In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-16 20:49:34--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-16 20:49:34 (9.17 MB/s) - ‘minsearch.py.1’ saved [4073/4073]



In [35]:
import minsearch

In [36]:
import json

In [37]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [38]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [39]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [40]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [41]:
#SELECT * WHERE course='data_engineering-zoomcamp';

In [42]:
q = 'the course has already started, can I stil enroll?'

In [43]:
index.fit(documents)

In [44]:
#boost = {'question': 3.0, 'section': 0.5}


#results = index.search(
#    query=q,
#    #filter_dict={'course': 'data-engineering-zoomcamp'},
#    boost_dict = boost,
#    num_results = 5
#)    

In [45]:
#results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoom

In [46]:
from openai import OpenAI


In [47]:
from dotenv import load_dotenv
import os

load_dotenv()
print("Key loaded:", "OPENAI_API_KEY" in os.environ)

Key loaded: True


In [48]:
client = OpenAI()

In [49]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on the specific course, institution, and their policies. Here are a few steps you can take:\n\n1. **Check the Course Website or Portal:** Some online platforms allow late enrollment within a certain timeframe. Look for any information about enrollment deadlines or late registration.\n\n2. **Contact the Instructor or Administration:** Reach out directly to the course instructor or the institution's administration. They might make exceptions or offer advice on how to proceed.\n\n3. **Consider Audit Options:** Some courses allow you to audit, meaning you can participate without receiving credit. This might be a good option if you cannot officially enroll.\n\n4. **Look for Other Sessions:** Sometimes, courses are offered multiple times throughout the year. Check if another session fits your schedule.\n\n5. **Review Policies:** Familiarize yourself with the institution's add/drop or late enrollment policies, as there mig

In [58]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only thefacts fromt he CONTEXT when answer the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [59]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [60]:
prompt = prompt_template.format(question=q, context=context).strip()

In [62]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only thefacts fromt he CONTEXT when answer the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I stil enroll?

CONTEXT: section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can

In [63]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course. In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'